This is the simulation step needed to handle driving
(small) distances left and right from an initial position and 
heading (a state). A state consists of the x,y positions of the
left drive wheel and the right drive wheel along with a heading.

Driving distances less than 1% of the wheelbase probably 
qualify as "small". 0.1% and 0.01% give better accuracy If driving distances are larger than that they should 
be divided into smaller steps.

This implementation treats clockwise turning as positive angles.

This is a super-simple model of robot physics: the robot
(or rather a robot drive wheel) goes at a speed determined by
the throttle setting. It does not take into account time
required to accelerate/decelerate to that speed; speed changes
are instantaneous. This might not be too bad if throttle changes
themselves cannot be instantaneous.

TODO
* Add acceleration modeling
* Add angular acceleration modeling
* Add tracjectory following (254 does this as part of their robot, not part of auto)
* Write auto programs for robot characterization
** speed vs throttle
** acceleration vs delta throttle
** angular rates vs throttle settings (not sure fully what this means yet)
** angular acceleration?
* Think about physical modeling of motors, gear ratios, robot mass, etc (I think this                                                                     might be better done outside the simulator                                                                        characterizing the results as acceleration and                                                                        velocity vs throttle values)
* Think about including the dead-reckoning position as part of the 
real robot state. If accurate enough it could help with some navigation
tasks.
* Sequence programs are destroyed by executing them; fix this!
* I think there are inconsistencies in how many levels of evaluation are needed between the different primitives. Rethink
* Rethink using implicit closures to represent programs; objects might be a little easier to comprehend

Important conventions
* Angles in radians
* Positive angles are counterclockwise
* Counters measure in distance units (not ticks)

In [ ]:
import math

maxSpeed = 100 # inches per second
turbo = 2 # maxSpeed scaled by this when turbo enabled
throttleDeadband = 0.2 # 20% of range
def sanitize(heading):
    while heading < 0: 
        heading += math.pi * 2
    while heading > (math.pi * 2):
        heading -= math.pi * 2
    return heading
# This was a bad idea; for programmed control we don't
# want a deadband for the throttles (though it is 
# probably a good idea for manual control)
# The problem is that it messes up linearity for the
# control algorithms and makes the control code more
# difficult
'''
def speedForThrottle(throttle, turbo=False):
    if throttle < 0: 
        sign = -1
    else: 
        sign = 1
    if turbo:
        turboScale = 2
    else:
        turboScale = 1
    absThrottle = abs(throttle)
    if absThrottle < 0.2: return 0
    elif absThrottle > 1: return sign * turboScale * maxSpeed
    else:
        return sign * turboScale * (maxSpeed *
           (absThrottle-throttleDeadband)/(1.0-throttleDeadband))
'''
def speedForThrottle(throttle):
    return(throttle*maxSpeed)

# newPos is for accumulating small deltas of movement 
smallThreshold = .001
# the smallThreshold value is what fraction of the wheelbase is
# considered "small" and is important to avoid accumulating too much
# error as a result of the discrete steps of integration
def newPos(wb,left, right, state):
    (heading, xLeft, yLeft, xRight, yRight) = state
    cos = math.cos(heading)
    sin = math.sin(heading)
    xLeft = xLeft + left*cos
    yLeft = yLeft + left*sin
    xRight = xRight + right*cos
    yRight = yRight + right*sin
    heading = heading + right/wb - left/wb
    if heading > 2*math.pi: heading -= 2*math.pi
    if heading < 0: heading += 2*math.pi
    return (heading, xLeft, yLeft, xRight, yRight)

# if either of the amounts of movement is not "small" then
# we need to break up the step into smaller increments
# newPosLarge handles this if needed and should be used 
# by any calling code (such as for the real robot which will
# need to use the integration method for dead reckoning)
# As a matter of software engineering it would be attractive
# to make newPos local to newPosLarge, but it also handy to have
# it available at top level for testing
def newPosLarge(wb, left, right, state):
    if (left/wb < smallThreshold) and (right/wb < smallThreshold):
        return newPos(wb, left, right, state)
    # make individual increments "small"
    scale = math.ceil(max(left/wb, right/wb) / smallThreshold)
    for i in range(scale):
        state = newPos(wb, left/scale, right/scale, state)
    return state

# For the simulator we can be much more efficient and much more
# accurate by computing the delta positions using a closed form
# calculation rather than relying on integration of small steps.
# Thus, the simulator should use newPosClosedForm rather than
# newPosLarge. These newPos functions might be included as 
# methods of the Robot, but having them as standalone allows 
# easier testing at the cost of passing some additional parameters
def newPosClosedForm(wb,left, right, state):
    (heading, xLeft, yLeft, xRight, yRight) = state
    if left==right:
        # drive straight ahead case; avoid division by 0
        cos = math.cos(heading)
        sin = math.sin(heading)
        xLeft = xLeft + left*cos
        yLeft = yLeft + left*sin
        xRight = xRight + right*cos
        yRight = yRight + right*sin
        return (heading, xLeft, yLeft, xRight, yRight)
    # otherwise it is a curve
    def deltaXY(initialAngle, radius, deltaAngle):
        xStart, yStart = math.cos(initialAngle), math.sin(initialAngle)
        finalAngle = initialAngle + deltaAngle
        # print(initialAngle, deltaAngle)
        xFinish, yFinish = math.cos(finalAngle), math.sin(finalAngle)
        deltaX, deltaY = radius*(xFinish-xStart), radius*(yFinish-yStart)
        return (deltaX, deltaY)
    innerDistance = min(abs(left), abs(right))
    outerDistance = max(abs(left), abs(right))
    if left*right<0: innerDistance *= -1
    outerRadius = wb/(1-innerDistance/outerDistance)
    innerRadius = outerRadius - wb
    deltaAngle = outerDistance/outerRadius
    if abs(right) > abs(left): 
        # right is outer
        rightRadius, leftRadius = outerRadius, abs(innerRadius)
        rightInitialAngle = heading - math.pi/2
        if innerRadius>=0:
            leftInitialAngle = rightInitialAngle
        else:
            leftInitialAngle = heading + math.pi/2
    else: 
        # left is outer
        leftRadius, rightRadius = outerRadius, abs(innerRadius)
        leftInitialAngle = heading + math.pi/2
        if innerRadius>=0:
            rightInitialAngle = leftInitialAngle
        else:
            rightInitialAngle = heading - math.pi/2
    if right > left:
        # positive deltaAngle
        pass
    else:
        deltaAngle *= -1
    #print(leftRadius, rightRadius, deltaAngle)
    deltaX, deltaY = deltaXY(leftInitialAngle, leftRadius, deltaAngle)
    xLeft, yLeft = xLeft + deltaX, yLeft + deltaY
    deltaX, deltaY = deltaXY(rightInitialAngle, rightRadius, deltaAngle)
    xRight, yRight = xRight + deltaX, yRight + deltaY
    heading += deltaAngle
    heading = sanitize(heading)
    return (heading, xLeft, yLeft, xRight, yRight)

def driveArc(xc, yc, xd, yd, heading):
    # Now we need to find the radius and distance to drive from current point
    # xc, yc to destination xd, yd around a circle when the heading at the current
    # point is theta. 
    # To do this we will find the perpendicular bisector of (xc,yc)->(xd,yd) and 
    # find the center of the circle where that line intersects the radius which will
    # be perpendicular to theta.
    # We can show that the angle, theta, between the radius at xc,yc and the perpendicular
    # bisector is equal to the angle between the heading and (xc,yc)->(xd,yd).
    # Knowing the length of (xc,yc)->(xd,yd) will then let us calculate the radius.


    # angle of (xc,yc)->(xd,yd)
    directAngle = atan2(yd-yc,xd-yc)
    deltaAngle = heading-directAngle
    if deltaAngle > pi: deltaAngle -= 2*pi
    if deltaAngle < -pi: deltaAngle += 2*pi
    theta = abs(deltaAngle)
    directDistance = sqrt((yd-yc)**2+(xd-xc)**2)
    if theta==0:
        # important special case to avoid division by 0
        return (0, 0, directDistance, heading) # 0: drive straight, 0: don't care, directDistance
    radius = (directDistance/2)/sin(theta)
    driveDistance = 2*theta*radius
    if deltaAngle>0:
        return (1, radius, driveDistance, heading+2*deltaAngle)
    else:
        return (-1, radius, driveDistance, heading+2*deltaAngle)


class Robot:
    state = (0, 0, 0, 0, 0) # heading leftX leftY rightX rightY
    leftCounter = 0
    rightCounter = 0
    leftThrottle = 0
    rightThrottle = 0
    clock = 0
    def __init__(self, wb=28.0, maxSpeed=60, turbo=2, deadband=0.2):
        self.wb, self.maxSpeed, self.turbo, self.deadband = (
            wb, maxSpeed, turbo, deadband)
        self.setPosition((0, 18, 0)) # position for a 36-inch long robot at the wall
    def getPosition(self):
        # tuple consisting of the heading, x, y positions of the 
        # center of the axle
        (heading, leftX, leftY, rightX, rightY) = self.state
        return ((heading, (leftX+rightX)/2.0, 
                          (leftY+rightY)/2.0))
    def setPosition(self, position):
        (heading, x, y) = position
        wb = self.wb
        sin = math.sin(heading)
        cos = math.cos(heading)
        rightX = x + wb/2 * sin
        rightY = y - wb/2 * cos
        leftX = x - wb/2 * sin
        leftY = y + wb/2 * cos
        print(f'{(heading, leftX, leftY, rightX, rightY)}')
        self.state = (heading, leftX, leftY, rightX, rightY)
        self.leftCounter, self.rightCounter = 0, 0
        
    def setThrottles(self, throttles):
        self.leftThrottle, self.rightThrottle = throttles
        
    def getThrottles(self):
        return (self.leftThrottle, self.rightThrottle)
        
    def getCounters(self):
        return (self.leftCounter, self.rightCounter)
    
    def impliedWheelbase(self):
        (heading, leftX, leftY, rightX, rightY) = self.state
        return math.sqrt((leftX-rightX)**2 
                         + (leftY-rightY)**2)
    def wheelbaseError(self):
        # how much do the wheel positions differ from the 
        # declared wheelbase
        # follow the convention that truth = measurement + error
        return self.wb - self.impliedWheelbase()
    def impliedHeading(self):
        # heading implied by the wheel positions
        # get the heading for lXY, rXY using atan2
        # use impliedWheelbase so cos and sin are consistent
        wb = self.impliedWheelbase()
        (heading, leftX, leftY, rightX, rightY) = self.state
        sin = (rightX-leftX)/wb
        cos = (leftY-rightY)/wb
        impliedHeading = math.atan2(sin,cos)
        return sanitize(impliedHeading)
    def headingError(self):
        (heading, leftX, leftY, rightX, rightY) = self.state
        # how much does the recorded heading differ from the 
        return heading - self.impliedHeading()
        
    def __repr__(self):
        (heading, leftX, leftY, rightX, rightY) = self.state
        return (f'{self.__class__.__name__}('
                f'leftXY: ({leftX},{leftY}), '
                f'rightXY: ({rightX},{rightY}), '
                f'heading: {heading}, '
                f'counters: ({self.leftCounter}, {self.rightCounter}), '
                f'wbError: {self.wheelbaseError()}, '
                f'headingError: {self.headingError()})'
                )
    def drive(self, lDist, rDist):
        self.state = newPosClosedForm(self.wb, lDist, rDist, self.state)
        self.leftCounter += lDist
        self.rightCounter += rDist
        
    def driveTimed(self, time):
        lThrot = self.leftThrottle
        rThrot = self.rightThrottle
        self.drive(speedForThrottle(lThrot)*time, speedForThrottle(rThrot)*time)
        
        
            

In [ ]:
from vpython import *



In [ ]:
from time import time
timeTime = time

DONE = 0
CONTINUE = 1
BADPROGRAM = -1

class Field:
    def __init__(self, leftOrRight): # 0 for left, 1 for right
        self.leftOrRight = leftOrRight
    def rightLeftChoice(self):
        return self.leftOrRight
class Auto:
    def __init__(self, field, robot, robotRep):
        self.field = field
        self.robot = robot
        self.robotRep = robotRep
        
    # for negative distance be sure to use negative throttle
    def driveStraightForDistance(self, distance, throttle):
        def init():
            (lc, rc) = self.robot.getCounters()
            lcLimit = lc + distance
            rcLimit = rc + distance
            print(throttle, lcLimit, rcLimit)
            robot.setThrottles((throttle, throttle))
            startClock = self.robot.clock
            def runStep(time):
                (lc, rc) = self.robot.getCounters()
                if (distance<0) and ((lc<=lcLimit) or (rc<=rcLimit)):
                    print('driveStraight took:', self.robot.clock-startClock)
                    return DONE
                elif (distance>=0) and ((lc>=lcLimit) or (rc>=rcLimit)):
                    print('driveStraight took:', self.robot.clock-startClock)
                    return DONE
                robot.driveTimed(time)
                return CONTINUE
            return runStep
        return init

    def zeroPtTurnToHeading(self, heading, throttle):
        def init():
            halfTurn = math.pi
            targetHeading = sanitize(heading)
            (initialHeading, rX, rY) = self.robot.getPosition()
            initialHeading = sanitize(initialHeading)
            headingDiff = targetHeading - initialHeading
            if headingDiff > halfTurn:
                headingDiff -= 2*halfTurn
            elif headingDiff < -halfTurn:
                headingDiff += 2*halfTurn
            if headingDiff > 0: 
                robot.setThrottles((-throttle, +throttle))
            else:
                robot.setThrottles((+throttle, -throttle))
            startClock = self.robot.clock
            def runStep(time):
                (robotHeading, rX, rY) = self.robot.getPosition()
                newHeadingDiff = targetHeading - robotHeading
                if newHeadingDiff > halfTurn:
                    newHeadingDiff -= 2*halfTurn
                elif newHeadingDiff < -halfTurn:
                    newHeadingDiff += 2*halfTurn
                if newHeadingDiff * headingDiff < 0: # the sign has changed
                    print('zeroPointTurn took:', self.robot.clock-startClock)
                    return DONE
                else:
                    robot.driveTimed(time)
                    return CONTINUE
            return runStep
        return init
    # radius of the path followed by the center of the robot
    # radius must be >=0; radius == 0 is a zero-point turn
    # radius == wb/2 is a 1-point turn; note that turns with a 
    # radius less than wb/2 require clearance for one side of the 
    # robot to move backward
    def arcToHeading(self, heading, throttle, radius):
        def init():
            halfTurn = math.pi
            targetHeading = sanitize(heading)
            (initialHeading, rX, rY) = self.robot.getPosition()
            initialHeading = sanitize(initialHeading)
            headingDiff = targetHeading - initialHeading
            print ("arcToHeading: initial heading, target heading: ", initialHeading, targetHeading)
            if headingDiff > halfTurn:
                headingDiff -= 2*halfTurn
            elif headingDiff < -halfTurn:
                headingDiff += 2*halfTurn
            outerRadius = radius + self.robot.wb/2
            throttleRatio = 1 - self.robot.wb/outerRadius
            # assert: -1 <= throttleRatio < 1
            innerThrottle = throttle * throttleRatio
            if headingDiff > 0: 
                robot.setThrottles((innerThrottle, throttle))
            else:
                robot.setThrottles((throttle, innerThrottle))
            startClock = self.robot.clock
            def runStep(time):
                (robotHeading, rX, rY) = self.robot.getPosition()
                newHeadingDiff = targetHeading - robotHeading
                if newHeadingDiff > halfTurn:
                    newHeadingDiff -= 2*halfTurn
                elif newHeadingDiff < -halfTurn:
                    newHeadingDiff += 2*halfTurn
                if newHeadingDiff * headingDiff < 0: # the sign has changed
                    print('Turn took:', self.robot.clock-startClock)
                    return DONE
                else:
                    robot.driveTimed(time)
                    return CONTINUE
            return runStep
        return init
              
    def driveToLocation(self, x, y, throttle):
        def init():
            robotHeading, rX, rY = self.robot.getPosition()
            dx = x-rX
            dy = y-rY
            newHeading = sanitize(atan2(dy,dx))
            robotHeading = sanitize(robotHeading)
            print(newHeading, robotHeading)
            distance = math.sqrt(dx*dx+dy*dy)
            actualThrottle = throttle
            #if abs(newHeading-robotHeading)>math.pi/2:
            #    newHeading -= math.pi
            #    distance = -distance
            #   actualThrottle = -throttle
            print ("driveToLocation zero point turn: ", robotHeading, newHeading, actualThrottle)
            p1 = self.arcToHeading(newHeading, throttle, 0) # zero point turn
            (leftStraightRight, radius, distance, finalHeading) = driveArc(
                rX, rY, x, y, newHeading)
            newHeading = sanitize(newHeading)
            print ("driveToLocation: ", newHeading, finalHeading, distance)
            if newHeading==finalHeading:
                p2 = self.driveStraightForDistance(distance, throttle)
            # elif newHeading==-finalHeading:
            #    p2 = self.driveStraightForDistance(distance, -throttle)
            else:
                # if abs(newHeading-finalHeading) < pi/2: 
                #    finalHeading = sanitize(finalHeading+pi)
                p2 = self.arcToHeading(finalHeading, radius, throttle)
            return self.runProg([p1,p2])
        return init
        
    def delay(self, interval):
        def init():
            stopTime = self.robot.clock+interval
            def run(time):
                if self.robot.clock >= stopTime:
                    return DONE
                else: return CONTINUE
            return run
        return init
    
    def runSequence(self, steps):
        #print("first of sequence")
        if not steps: return None
        steps[0] = self.runProg(steps[0])
        def run(time):
            res = steps[0](time)
            if res==DONE:
                #print("finished sequence element")
                #print(f'{self.robot}')
                del steps[0]
                if not steps: 
                    #print("finished sequence")
                    return DONE
                steps[0] = self.runProg(steps[0])
            return CONTINUE
        return run
    
    def runCase(self, prog):
        test, cases = prog
        res = test()
        #print(f'Case test result {res}')
        return self.runProg(cases[test()])
        
    def runProg(self, prog):
        # must return an "run" function taking a time and itself returning
        # None or a run function
        #print(f'Running prog {prog}')
        if type(prog)==list:
            return self.runSequence(prog)
        elif type(prog)==tuple:
            return self.runCase(prog)
        else:
            return prog()
            
    def centerToLeftOrRight(self):
        prog = [self.delay(2),
                (self.field.rightLeftChoice, 
                     [self.arcToHeading(-math.pi/4, 1, 14),
                      self.arcToHeading(math.pi/4, 1, 14),
                     ]), 
                self.driveStraightForDistance(73, 1),
                self.arcToHeading(0, 1, 28),
                self.driveStraightForDistance(40, 1),
                self.driveStraightForDistance (10, 0.8),
               ] 
        return prog
    
    def leftToLeftOrRight(self):
        leftSideToRightSide = [
                self.driveStraightForDistance(19.5*feet, 1),
                self.arcToHeading(-math.pi/2, 1, 14),
                self.driveStraightForDistance(18.5*feet, 1),
                self.arcToHeading(-math.pi, 1, 14),
                self.driveStraightForDistance(36,1),
                self.arcToHeading(math.pi/2, 1, 14),
                self.driveStraightForDistance(12, 1),
               ]
                 
        leftSideToLeftSide = [
            self.driveStraightForDistance(14.5*feet, 1),
            self.arcToHeading(-math.pi/2, 1, 14),
            self.driveStraightForDistance(23, 0.8)
        ]
        prog = (self.field.rightLeftChoice,
                   [leftSideToLeftSide,
                   leftSideToRightSide])
        return prog
        
        
    def auto(self, prog):
        startTime = timeTime()
        time = 0
        robot.clock = time
        steps = 0
        timeStep = 0.01
        run = self.runProg(prog)
        while True:
            res = run(timeStep)
            if res == DONE: 
                break
            rate(100) # this must be the sleep from vpython and NOT time.sleep
            time += timeStep
            steps += 1
            robot.clock = time
            (heading, X, Y) = self.robot.getPosition()
            self.robotRep.pos = vector(X, Y, 3)
            self.robotRep.axis = vector(cos(heading)*36, sin(heading)*36, 0)
        print(f'{robot}\n')
        print(f'Simulation took {timeTime()-startTime} seconds and {steps} steps\n')
                

In [ ]:
from vpython import *
scene = canvas()

In [ ]:
feet = 12
inches = 1
gray = vector(0.5,0.5,0.5)
field = box(size=vector(27*feet, 27*feet, 1), 
              pos=vector(27/2*feet,0,-1), color=gray)
robotRep = box(size=vector(36, 28, 6), pos=vector(18,0,3),
            color=color.blue)
switch = box(size=vector(4*feet+8*inches, 12*feet+9.5*inches, 1*feet+6.75*inches),
             pos=vector(14*feet+(4*feet+8*inches)/2, 0, (1*feet+6.75*inches)/2), color=color.red)
cubepile1 = box(size=vector(1*feet, 3*feet, 1*feet), 
                pos=vector(14*feet-0.5*feet, 0, 0.5*feet), color=color.yellow)
cubepile2 = box(size=vector(1*feet, 2*feet, 1*feet),
                pos=vector(14*feet-1.5*feet, 0, 0.5*feet), color=color.yellow)
cubepile3 = box(size=vector(1*feet, 1*feet, 1*feet),
                pos=vector(14*feet-2.5*feet, 0, 0.5*feet), color=color.yellow)
cubepile4 = box(size=vector(1*feet, 1*feet, 1*feet),
                pos=vector(14*feet+4*feet+8*inches+0.5*feet, 0, 0.5*feet), color=color.yellow)
scene.userzoom = False 
scene.userspin = False 
scene.userpan = False 
scene.center = vector(13.5*feet,0,0)
scene.center=vector(122,-41,0)
scene.forward=vector(0.2, 0.0,-0.6)
scene.up = vector(0,0,1)
robot = Robot()
leftAuto = Auto(Field(0), robot, robotRep)
rightAuto = Auto(Field(1), robot, robotRep)

In [ ]:
scene.userzoom = True
scene.userspin = True
scene.userpan = True

In [ ]:
# Demo of Auto Programs -- (left and center start) to (left and right); arbitrary point and heading to Exchange
# autos = [leftAuto, rightAuto] # skip these while debugging other code
autos = [] 
while True:
    for auto in autos:
        for startPos in [0,1]:
            sleep(2)
            if startPos == 0:
                prog = auto.centerToLeftOrRight()
                robot.setPosition((0, 18, 0))
                robotRep.pos = vector(18, 0, 3)
            else:
                prog = auto.leftToLeftOrRight()
                robot.setPosition((0, 18, 11*feet))
                robotRep.pos = vector(18, 11*feet, 3)
            auto.auto(prog)
    sleep(2)
    prog = [leftAuto.delay(2), 
        leftAuto.driveToLocation(40, 0, 1.0), # move to 'staging' point
        leftAuto.arcToHeading(pi, 1, 0),  # make sure facing the right direction      
        leftAuto.driveToLocation(30, 0, 0.6), # move to second staging point
        leftAuto.arcToHeading(0, 0.4, 0), # fine tune the direction
        leftAuto.driveStraightForDistance(-6, -0.6)]
    robot.setPosition((-math.pi/3, 8*feet, 8*feet))
    robotRep.pos = vector(8*feet, 8*feet, 3)
    leftAuto.auto(prog)
    sleep(2)
    prog = [leftAuto.delay(2), 
        leftAuto.driveToLocation(40, 0, 1.0), # move to 'staging' point
        leftAuto.arcToHeading(pi, 1, 0),  # make sure facing the right direction      
        leftAuto.driveToLocation(30, 0, 0.6), # move to second staging point
        leftAuto.arcToHeading(0, 0.4, 0), # fine tune the direction
        leftAuto.driveStraightForDistance(-6, -0.6)]
    robot.setPosition((-math.pi/3, 8*feet, -8*feet))
    robotRep.pos = vector(8*feet, -8*feet, 3)
    leftAuto.auto(prog)

In [ ]:
print(robot)